In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver

In [59]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://lewan.baidu.com/rankland?idfrom=5056&gameSource=standalone')
time.sleep(3)
game_list = []
num = 0

# 장르 번역작업 및 통합을 위한 리스트
genre_chi = ['独立游戏', '动作', '角色扮演', '策略经营', '休闲', '射击', '益智', '赛车', '体育', '桌游棋牌', '音舞', '其它', '策略经营', '模拟', '格斗', '开放世界', 'ARPG', '脑力', '战略', '枪械', '传统RPG', '冒险', '卷轴动作', 'MMORPG', '竞速']
genre_kor = ['인디', '액션', 'RPG', '전략', '퍼즐', '슈팅', '퍼즐', '레이싱', '스포츠', '보드', '리듬', '기타', '전략', '시뮬레이션', '격투', '오픈월드', 'RPG', '퍼즐', '전략', '슈팅', 'RPG', '어드벤쳐', '플랫폼', 'RPG', '레이싱']

# 원하는 페이지 수만큼 데이터 추출
for i in range(77):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    a_list = soup.find_all(class_ = 'info-box')
    for item in a_list:
        num += 1
# 게임제목을 되도록이면 영문명으로 추출하되 영문명이 없을경우 중국어로 추출
        title = item.find(class_="ename desc").text.replace(',','.') 
        if not title:
            title = item.find(class_="cname").text.replace(',','.')
# 출시 연도를 추출하되 연도가없는경우 오류가 발생하므로 None으로 추출
        try :
            date = item.find(class_="publish-time").text[:4]
        except :
            date = None
        genre = item.find(class_="type").text.split("/")
        for ge in genre: # 장르 번역작업
            if ge in genre_chi:
                genre[genre.index(ge)] = genre_kor[genre_chi.index(ge)]
        rank = num
        game_list.append([rank,title, date, genre])
# 페이지를 자동으로 넘기기 위해 클릭코드 삽입
    driver.find_element_by_css_selector('#root > div > div > div.layout-main > div > div > div.main-area > div.top-area-box > div.top-area > div > div.type-rank-main > div.right > ul > li.rc-pagination-next').click()  
    time.sleep(2)

game_list

[[1, 'Plants vs. Zombies', '2009', ['퍼즐', '퍼즐']],
 [2, 'Minecraft', '2014', ['전략', '오픈월드']],
 [3, 'Terraria', '2011', ['인디']],
 [4, 'Command ', '1996', ['전략', '전략']],
 [5, 'Call Of Duty', '2016', ['슈팅', '슈팅']],
 [6, 'Red Dead Redemption Ⅱ', '2018', ['액션', '어드벤쳐']],
 [7, 'Grand Theft Auto', '1998', ['RPG', '기타']],
 [8, 'Counter-Strike', '2000', ['슈팅', '슈팅']],
 [9, 'ARK: Survival Evolved', None, ['전략', '오픈월드']],
 [10, "Sid Meier's Civilization VI", '2019', ['전략', '전략']],
 [11, 'legend of mir 2', None, ['RPG', 'RPG']],
 [12, 'NIOH', '2017', ['액션', '기타']],
 [13, 'Battlefield 5', None, ['슈팅', '슈팅']],
 [14, 'Minesweeper', None, ['퍼즐', '퍼즐']],
 [15, 'Grand Theft Auto：Vice City', '2003', ['전략', '오픈월드']],
 [16, 'Diablo 2', '2000', ['RPG', 'RPG']],
 [17, 'Stardew Valley', '2016', ['전략', '시뮬레이션']],
 [18, 'Don’t Starve', '2014', ['전략', '养成']],
 [19, 'The Witcher 3: Wild Hunt', '2015', ['RPG', 'RPG']],
 [20, 'Warcraft', None, ['전략', '전략']],
 [21, 'The Sims Ⅳ', '2014', ['전략', '시뮬레이션']],
 [22, 'THE K

In [60]:
# 장르수 최대값 구하기 및 장르 분리화 작업
maxlen = 0
game_list2 = []
for item in game_list:
    if len(item[3]) > maxlen:
        maxlen = len(item[3])
for game in game_list:
    info = []
    info.append(game[0])
    info.append(game[1])
    info.append(game[2])
    for genre in game[3]:
        if genre not in info:
            info.append(genre)
    game_list2.append(info)
for list in game_list2:
    if len(list) < 3+maxlen :
        for i in range(3+maxlen - len(list)):
            list.append(None)

frame2 = ['순위', '제목', '출시날짜']
for i in range(maxlen):
    frame2.append('장르'+str(i+1))
df = pd.DataFrame(game_list2, columns=frame2)
df

,순위,제목,출시날짜,장르1,장르2
0,1,Plants vs. Zombies,2009,퍼즐,None
1,2,Minecraft,2014,전략,오픈월드
2,3,Terraria,2011,인디,None
3,4,Command,1996,전략,None
4,5,Call Of Duty,2016,슈팅,None
...,...,...,...,...,...
765,766,The Town of Light,2016,RPG,기타
766,767,The Colonists,2018,전략,시뮬레이션
767,768,宝可梦：漆黑的魅影,2016,RPG,None
768,769,Project Hospital,2018,전략,시뮬레이션


In [61]:
# csv로 추출
df.to_csv('project/console_chi_data.csv', index=False)